In [ ]:
# testing of spend merchant location data
import pandas as pd
from google.cloud import bigquery, storage
from pyarrow import *

In [ ]:
# summary stats of production file
%bigquery_stats ons-fintrans-data-prod.fintrans_visa.spend_merchant_location

In [ ]:
%%bigquery  
CREATE OR REPLACE VIEW ons-fintrans-analysis-prod.fin_wip_notebook.spend_merchant_location_unique_mcc_AH AS 
SELECT mcc, mcg, 
  SUM(spend) as sum_spend, 
  SUM(transactions) as sum_transactions, 
  SUM(cardholders) as sum_cardholders, 
  SUM(dist_merchants) as sum_dist_merchants, 
  AVG(pct_repeat_pan_cnt) as avg_pct_repeat_pan_cnt
FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
WHERE merchant_location_level = "All"  
  AND cardholder_issuing_level = "Domestic"
  AND time_period = "Quarter"
  AND mcc <> "All"
GROUP BY mcc, mcg; 

In [ ]:
%bigquery_stats ons-fintrans-analysis-prod.fin_wip_notebook.spend_merchant_location_unique_mcc_AH

In [ ]:
# read in derivative WIP file function
def read_data_from_bigquery_to_df(sql):
    """
    Gets data from BigQuery and saves to Pandas DataFrame

    Args:
       sql (str): the sql query to determine what data to return
    Returns:
       the query results in a Pandas dataframe , or None if error
    """
    try:
        client = bigquery.Client()
        df = client.query(sql).to_dataframe()
        return df
    except Exception as e:
        print(f"Error getting data {e}")
        return None

In [ ]:
# read in derivative WIP file
unique_spend = read_data_from_bigquery_to_df(
    "SELECT * FROM ons-fintrans-analysis-prod.fin_wip_notebook.spend_merchant_location_unique_mcc_AH"
)

In [ ]:
# summary stats
unique_spend.sample(10)

In [ ]:
# tests